![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
print(ds_jobs_transformed.shape)
print(ds_jobs_transformed.info())

print(ds_jobs_transformed.isna().sum())

# "training_hours" - 241 unique values (...), datatype is int64, There are no NA.
print(ds_jobs_transformed["training_hours"].value_counts(dropna=False))



(19158, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), o

In [3]:
# info of original df
print(ds_jobs_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [4]:
# Frequency of values in each column

# "student_id" - 19158 unique values, datatype is int64, There is no NA - Need to store it as 32-bit integers (int32)
print(ds_jobs_transformed["student_id"].value_counts(dropna=False))
print(ds_jobs_transformed["student_id"].dtype)
ds_jobs_transformed["student_id"] = ds_jobs_transformed["student_id"].astype('int32')
print(ds_jobs_transformed["student_id"].dtype)

8949     1
10660    1
30726    1
18507    1
31273    1
        ..
11547    1
32067    1
14356    1
18051    1
23834    1
Name: student_id, Length: 19158, dtype: int64
int64
int32


In [5]:
# "city" - 123 unique categories, datatype is object, There is no NA
print(ds_jobs_transformed["city"].value_counts(dropna=False))
ds_jobs_transformed["city"] = ds_jobs_transformed["city"].astype("category")
print(ds_jobs_transformed["city"].cat.categories)

city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64
Index(['city_1', 'city_10', 'city_100', 'city_101', 'city_102', 'city_103',
       'city_104', 'city_105', 'city_106', 'city_107',
       ...
       'city_84', 'city_89', 'city_9', 'city_90', 'city_91', 'city_93',
       'city_94', 'city_97', 'city_98', 'city_99'],
      dtype='object', length=123)


In [6]:
# if it is int64 set it as int32 - for all columns
columns = ds_jobs_transformed.columns
for column in ds_jobs_transformed[columns]:
    if ds_jobs_transformed[column].dtype == 'int64':
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype('int32')
        print(ds_jobs_transformed[column].dtype)

int32


In [7]:
# "city_development_index" - 93 unique categories, datatype is float64, There is no NA - Need to store it as 16-bit floats (float16)
print(ds_jobs_transformed["city_development_index"].value_counts(dropna=False))

0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.807       4
0.781       3
0.625       3
0.664       1
Name: city_development_index, Length: 93, dtype: int64


In [8]:
# if it is float64 set it as float16 - for all columns
columns = ds_jobs_transformed.columns
for column in ds_jobs_transformed[columns]:
    if ds_jobs_transformed[column].dtype == "float64":
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype("float16")
        print(ds_jobs_transformed[column].dtype)

float16
float16


In [9]:
# "gender" - 4 unique categories (Male, NaN, Female, other), datatype is object, There are some NA. - NOMINAL CATEGORICAL DATA - must be stored as the category data type
print(ds_jobs_transformed["gender"].value_counts(dropna=False))
print(ds_jobs_transformed["gender"].dtype)
ds_jobs_transformed["gender"] = ds_jobs_transformed["gender"].astype("category")
print(ds_jobs_transformed["gender"].dtype)

Male      13221
NaN        4508
Female     1238
Other       191
Name: gender, dtype: int64
object
category


In [10]:
# "enrolled_university" - 4 unique categories (no_enrollment, Full-time course, Part-time course, NaN), datatype is object, There are some NA. - ORDINAl CATEGORICAL DATA - must be stored as the category data type
print(ds_jobs_transformed["enrolled_university"].value_counts(dropna=False))
print(ds_jobs_transformed["enrolled_university"].dtype)
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].astype("category")
print(ds_jobs_transformed["enrolled_university"].cat.categories)
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].cat.reorder_categories(new_categories=["no_enrollment", "Part time course", "Full time course"], ordered=True)
print(ds_jobs_transformed["enrolled_university"].cat.categories)

print(ds_jobs_transformed["enrolled_university"].dtype)

no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64
object
Index(['Full time course', 'Part time course', 'no_enrollment'], dtype='object')
Index(['no_enrollment', 'Part time course', 'Full time course'], dtype='object')
category


In [11]:
# "major_discipline" - 7 unique categories (STEM, NaN, Humanities, Other, Business Degree, Arts, No Major), datatype is object, There are some NA. - NOMINAL CATEGORICAL DATA - must be stored as the category data type
print(ds_jobs_transformed["major_discipline"].value_counts(dropna=False))
print(ds_jobs_transformed["major_discipline"].dtype)
ds_jobs_transformed["major_discipline"] = ds_jobs_transformed["major_discipline"].astype("category")
print(ds_jobs_transformed["major_discipline"].dtype)

STEM               14492
NaN                 2813
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64
object
category


In [12]:
# "company_type" - 7 unique categories (...), datatype is object, There are some NA. - NOMINAL CATEGORICAL DATA - must be stored as category data type
print(ds_jobs_transformed["company_type"].value_counts(dropna=False))
print(ds_jobs_transformed["company_type"].dtype)
ds_jobs_transformed["company_type"] = ds_jobs_transformed["company_type"].astype("category")
print(ds_jobs_transformed["company_type"].dtype)

Pvt Ltd                9817
NaN                    6140
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: company_type, dtype: int64
object
category


In [13]:
# "education_level" - 6 unique categories (Graduate, Masters, High School, NaN, Phd, Primary School), datatype is object, There are some NA. - ORDINAL CATEGORICAL DATA - need to order
print(ds_jobs_transformed["education_level"].value_counts(dropna=False))
print(ds_jobs_transformed["education_level"].dtype)
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].astype("category") 
print(ds_jobs_transformed["education_level"].dtype)

print(ds_jobs_transformed["education_level"].cat.categories)
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].cat.reorder_categories(new_categories = ["Primary School", "High School", "Graduate", "Masters", "Phd"], ordered=True)
print(ds_jobs_transformed["education_level"].cat.categories)

Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: education_level, dtype: int64
object
category
Index(['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], dtype='object')
Index(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], dtype='object')


In [14]:
# "experience " - 23 unique categories (...), datatype is object, There are some NA. - ORDINAL CATEGORICAL DATA - need to order

print(ds_jobs_transformed["experience"].value_counts(dropna=False))
print(ds_jobs_transformed["experience"].dtype)
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].astype("category")
print(ds_jobs_transformed["experience"].dtype)
print(ds_jobs_transformed["experience"].cat.categories)
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].cat.reorder_categories(new_categories=["<1", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", ">20"], ordered=True)
print(ds_jobs_transformed["experience"].cat.categories)

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: experience, dtype: int64
object
category
Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '3', '4', '5', '6', '7', '8', '9', '<1', '>20'],
      dtype='object')
Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')


In [15]:
# "company_size" - 9 unique categories (...), datatype is object, There are some NA. - ORDINAL CATEGORICAL DATA - need to order

print(ds_jobs_transformed["company_size"].value_counts(dropna=False))

print(ds_jobs_transformed["company_size"].dtype)
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].astype("category")
print(ds_jobs_transformed["company_size"].dtype)

print(ds_jobs_transformed["company_size"].cat.categories)
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].cat.reorder_categories(new_categories=["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"], ordered=True)
print(ds_jobs_transformed["company_size"].cat.categories)


NaN          5938
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64
object
category
Index(['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999',
       '5000-9999', '<10'],
      dtype='object')
Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999',
       '10000+'],
      dtype='object')


In [16]:
# "last_new_job" - 7 unique categories (...), datatype is object, There are some NA. - ORDINAL CATEGORICAL DATA - need to order

print(ds_jobs_transformed["last_new_job"].value_counts(dropna=False))

print(ds_jobs_transformed["last_new_job"].dtype)
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].astype("category")
print(ds_jobs_transformed["last_new_job"].dtype)

print(ds_jobs_transformed["last_new_job"].cat.categories)
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].cat.reorder_categories(new_categories=["never", "1", "2", "3", "4", ">4"], ordered=True)
print(ds_jobs_transformed["last_new_job"].cat.categories)

1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
NaN       423
Name: last_new_job, dtype: int64
object
category
Index(['1', '2', '3', '4', '>4', 'never'], dtype='object')
Index(['never', '1', '2', '3', '4', '>4'], dtype='object')


In [17]:
# "relevant_experience" - 2 unique categories (Has relevant experience, No relevant experience), datatype is object, There is no NA. - must be stored as Booleans

print(ds_jobs_transformed["relevant_experience"].value_counts(dropna=False))

ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].map({"Has relevant experience": True, "No relevant experience": False})

print(ds_jobs_transformed["relevant_experience"].value_counts(dropna=False))


Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64
True     13792
False     5366
Name: relevant_experience, dtype: int64


In [18]:
# "job_change" - 2 unique values (0, 1), datatype is float64, There are no NA. - must be stored as Booleans

print(ds_jobs_transformed["job_change"].value_counts(dropna=False))

ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].map({1.0: True, 0.0: False})
print(ds_jobs_transformed["job_change"].value_counts(dropna=False))




0.0    14381
1.0     4777
Name: job_change, dtype: int64
False    14381
True      4777
Name: job_change, dtype: int64


In [19]:
print(ds_jobs_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [20]:
print(ds_jobs.memory_usage())
print(ds_jobs_transformed.memory_usage())

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64
Index                       128
student_id                76632
city                      24310
city_development_index    38316
gender                    19290
relevant_experience       19158
enrolled_university       19290
education_level           19370
major_discipline          19378
experience                19890
company_size              19522
company_type              19378
last_new_job              19378
training_hours            76632
job_change                19158
dtype: int64

In [21]:

print(ds_jobs_transformed["experience"].cat.categories)

# Create a new dictionary with ten_or_more_years of experience
ten_or_more_years = {"10": "10+", "11": "10+", "12": "10+", "13": "10+", "14": "10+", "15": "10+", "16": "10+", "17": "10+", "18": "10+", "19": "10+", "20": "10+", ">20": "10+"}

# Create a new column experience_collapsed
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].replace(ten_or_more_years)
print(ds_jobs_transformed["experience"].cat.categories)



print(ds_jobs_transformed["company_size"].cat.categories)

# Create a new dictionary with at_least_thousand_employees dictionary
at_least_thousand_employees = {"1000-4999": "1000+", "5000-9999": "1000+", "10000+": "1000+"}

# Create a new column company_size_collapsed
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].replace(at_least_thousand_employees)
print(ds_jobs_transformed["company_size"].cat.categories)

# Filter for 10 or more years of experience at companies with at least 1000 employees
company_size_thousand = ds_jobs_transformed["company_size"] == "1000+"

ten_years_experience = ds_jobs_transformed["experience"] == "10+"

ds_jobs_transformed = ds_jobs_transformed.loc[(ds_jobs_transformed["experience"] == "10+") & (ds_jobs_transformed["company_size"] == "1000+")]
print(ds_jobs_transformed.head())
print(ds_jobs_transformed.info())

Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')
Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+'], dtype='object')
Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999',
       '10000+'],
      dtype='object')
Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000+'], dtype='object')
    student_id      city  ...  training_hours job_change
9          699  city_103  ...             123      False
12       25619   city_61  ...              23      False
31       22293  city_103  ...             141      False
34       26494   city_16  ...             145      False
40        2547  city_114  ...              14      False

[5 rows x 14 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  -